In [1]:
import os

from pyspark import SparkConf
from pyspark.sql import SparkSession

# spark-sql --version
# setting environ variables essentially
# we're also installing? libraries straight by these commands rather than manually.
# https://hadoop.apache.org/docs/r2.8.0/hadoop-aws/tools/hadoop-aws/index.html#Changing_Authentication_Providers
conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.1') # basically most recent version as my spark version
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
conf.set('spark.hadoop.fs.s3a.access.key', os.environ.get('aws_access_key_id'))
conf.set('spark.hadoop.fs.s3a.secret.key', os.environ.get('aws_secret_access_key'))

# building it via a config rather than a raw .appName()
spark = SparkSession.builder.config(conf=conf).getOrCreate()

22/05/20 16:05:01 WARN Utils: Your hostname, jacob-BigOtisLinux resolves to a loopback address: 127.0.1.1; using 192.168.50.142 instead (on interface enp6s0)
22/05/20 16:05:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/jacob/.ivy2/cache
The jars for the packages stored in: /home/jacob/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-eb179d56-0bbd-433e-8fa1-37955b98419e;1.0
	confs: [default]


:: loading settings :: url = jar:file:/home/jacob/.local/share/virtualenvs/pyspark_prac-QhYfwHaC/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.hadoop#hadoop-aws;3.3.1 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.901 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 126ms :: artifacts dl 5ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.1 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0   ||   3   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-eb179d56-0bbd-433e-8fa1-37955b98419e
	confs: [default]


In [3]:
## STEP 1 Read file from S3
df = spark.read.parquet('s3a://jacobsbucket97/sample_files/boxscores-2022-02-26.parquet', inferSchema = True, header = True)
# df.printSchema()
# df2 = df.toPandas()
df.createTempView('nba_data')

22/05/20 16:05:34 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
## STEP 2 Transform the file and add or remove things
df5 = spark.sql("SELECT * FROM nba_data limit 5;")
df_spark = df5.collect() # will pull it into a python list

In [6]:
# STEP 3 Write it back to S3
s3_dest_path = "s3a://jacobsbucket97/pyspark/nba_tweets.csv"
df5.write.mode("overwrite").format("csv").save(s3_dest_path)
df5.write.parquet('s3a://jacobsbucket97/sample_files/nba_tweets_v2.parquet')

22/05/20 16:06:00 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/05/20 16:06:01 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [11]:
df3 = spark.read.parquet('s3a://jacobsbucket97/pyspark/nba_tweets_v2.parquet/part-00000-f4044b57-7cdf-4d1c-8065-80d54fa346cf-c000.snappy.parquet')
df3_pandas = df3.toPandas()